Step 1. 데이터 전처리
TfidfVectorizer로 텍스트 벡터화
Stopwords 제거, 소문자 통일 등 기본 클렌징

Step 2. 세 가지 모델 학습
모델 1: Naive Bayes (MultinomialNB)
모델 2: SGDClassifier (linear SVM with hinge loss)
모델 3: BERT 기반 분류기 (Hugging Face의 transformers 라이브러리 사용, distilbert-base-uncased 추천)

Step 3. 모델 학습 및 예측
훈련 데이터로 학습 → 테스트 데이터로 예측 수행

Step 4. 성능 비교
정확도(Accuracy)
정밀도/재현율/F1-score (macro average)
학습 속도 및 예측 시간 비교 (옵션)

Step 5. 결과 분석 및 보고서 작성
성능 비교 그래프/표 작성
각 모델의 강점/약점 분석

최종적으로 어떤 모델이 왜 적합한지 선정


In [1]:
# 0. 데이터셋 준비
# 이 코드는 20 Newsgroups 데이터셋을 불러오는 예제입니다.
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
import torch

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Step 1: 데이터 전처리
# TfidfVectorizer로 텍스트 벡터화

# Stopwords 제거, 소문자 통일 등 기본 클렌징